In [1]:
!pip install langchain langchain_core langchain_groq langchain_community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00


In [6]:
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
!pip install -qU duckduckgo-search langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.9 MB/s eta 0:00:00


In [7]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)

In [8]:
result = llm.invoke('What is apple')
result.content

"A simple yet delicious question!\n\nAn apple is a type of fruit that comes from the apple tree (Malus domestica). It is a juicy, sweet, and crunchy fruit that is often eaten fresh or used in a variety of culinary and cultural contexts.\n\nApples are typically round or oval in shape, with a thin skin that can range in color from red, green, yellow, to a combination of these colors. The flesh of the apple is firm and juicy, with a sweet and slightly tart taste.\n\nApples are a great source of fiber, vitamins, and antioxidants, making them a popular and healthy snack. They are also used in a variety of products, such as:\n\n1. Juice and cider\n2. Pies and baked goods\n3. Salads and savory dishes\n4. Dried fruit and fruit leather\n5. Apple sauce and butter\n\nThere are over 7,500 known varieties of apples worldwide, each with its unique characteristics, flavor profile, and texture. Some popular varieties include:\n\n1. Red Delicious\n2. Granny Smith\n3. Fuji\n4. Gala\n5. Golden Delicious\

In [9]:
@tool
def get_course_amount(input: str):
    """
    Get the course name from the input question and return the corresponding amount.
    """

    course_prices = {
        "Javascript": 5000,
        "Python": 8000,
        "Angular": 3000,
        "Data Science": 10000,
        "Gen AI": 15000
    }

    prompt = ChatPromptTemplate.from_template(
        "If the question contains any course name, extract only the course name. "
        "Example: If user asks 'What is the price of Reactjs course?', return only 'Reactjs'. "
        "Do NOT include additional text. "
        "Here is the student's question: {input}"
    )
    chain = prompt | llm
    response = chain.invoke({'input': input}).content

    course_amount = course_prices.get(response, "course is not found")
    return f"The price of {response} course is {course_amount}"

In [10]:
@tool
def search_from_browser(input: str):
  """
  If any thing has to be searched in the browser, then this function has to be used, as per to the question asked input search in browser and return the respone.
  """

  search = DuckDuckGoSearchRun()
  return search.invoke(input)

In [11]:
@tool
def multiply(x: float, y: float)-> float:
  """Multiply 'x' times 'y'."""
  return x * y

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [14]:
tools = [multiply, search_from_browser, get_course_amount]

In [15]:
agent = create_tool_calling_agent(llm, tools=tools, prompt=prompt)

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
agent_executor.invoke({"input": "please do multiplication of the 2 and 5?"})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 2, 'y': 5}`


10.0The result of multiplying 2 and 5 is 10.0.

> Finished chain.


{'input': 'please do multiplication of the 2 and 5?',
 'output': 'The result of multiplying 2 and 5 is 10.0.'}